### RankNet-makedb-pitmodel

make gluonts dataset for pitmodel training only

+ todo add inlap into lapstatus(the lap before 'P')

base: 18./make_gluonts_dataset-pitagemodel

+ set lap2nextpit as target, this is as the same as a stacked lstm model when set prediction_length=1
+ set lap_instint as target, track_status as covariate

base: 15./laptime_rank_timediff_dataset-oracle-fulltest-pitage

+ add lap_instint to dynamic features(pitage)
+ add support for historical data, indy500 for different years
+ add track and lap status as target, preparing for StatusModel with deepAR

this notebook focuses on create the training set, while laptime2rank-evaluate will have testset codes specified for rank calculation

+ add new features, caution_laps_instint,lap_instint

Build a time series dataset across all the oval races, including laptime, rank. When this dataset aims to be used in forecasting, covariates of the racing status can not be included, such as track_status and lap_status. However, they can be used in oracle test to tell the upper bound of performance of the predictor.

Change to a new dataset format that following telemetry dataset.

raw:
+  [(eventid, carids: carno -> rowid, datalist)]

datalist := [datalist_entry] in shape of #car_number

datalist_entry := [[laptime, rank, track_status, lap_status]], in shape of #totallaps x #featureCnt (padded by nan)

gluonts:

+ Rank2LaptimeDifference

rank prediction by elapsed time difference in a race

Since use laptime2rank is not very successful, and predicting rank directly is not as well, to combine the real value regression into rank prediction task might be a solution.

And, differences of the elapsed time among the cars takes correlations into input, that brings more chances to success.

time_behind_leader is accurate when the car runs in the same lap as the leader does, otherwise, the time is truncted. To get accurate differences, using elapsed time is needed.


In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import random
import mxnet as mx
from mxnet import gluon
import pickle
import json
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [3]:
import os
os.getcwd()

'/scratch_hdd/hpda/indycar/notebook/16.PitageModel'

In [12]:

def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

def test_flag(a, bitflag):
    return (a & bitflag) ==  bitflag

#
# remove NaN at the tail
# there should be no nans in the middle of the ts
COL_LAPTIME=0
COL_RANK=1
COL_TRACKSTATUS=2
COL_LAPSTATUS=3
COL_TIMEDIFF=4
COL_CAUTION_LAPS_INSTINT=5
COL_LAPS_INSTINT= 6
COL_ELAPSED_TIME= 7
COL_LAP2NEXTPIT = 8


MODE_ORACLE = 0
MODE_NOLAP = 1
MODE_NOTRACK = 2
MODE_TESTZERO = 4
MODE_TESTCURTRACK = 8
#MODE_STR={MODE_ORACLE:'oracle', MODE_NOLAP:'nolap',MODE_NOTRACK:'notrack',MODE_TEST:'test'}

def make_dataset_byevent(runs, prediction_length, freq, 
                       useeid = False,
                       run_ts=COL_LAPTIME, 
                       test_event = 'Indy500-2018',
                       use_global_dict = True,
                       oracle_mode = MODE_ORACLE,
                       half_moving_win = 0,
                       train_ratio=0.8,
                       log_transform = False,
                       context_ratio = 0.,
                       dorerank = True
                ):
    """
    split the ts to train and test part by the ratio
    
    oracle_mode: false to simulate prediction in real by 
        set the covariates of track and lap status as nan in the testset
            
    
    """    
    start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

    train_set = []
    test_set = []
    
    #select run
    if runs>=0:
        _laptime_data = [laptime_data[runs].copy()]
    else:
        _laptime_data = laptime_data.copy()
    
   
    #_data: eventid, carids, datalist[carnumbers, features, lapnumber]->[laptime, rank, track, lap]]
    for _data in _laptime_data:
        _train = []
        _test = []
        
        if events[_data[0]] == test_event:
            test_mode = True
        
        else:
            test_mode = False
            
            
        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        train_len = int(np.max(ts_len) * train_ratio)
        
        if context_ratio != 0.:
            # add this part to train set
            context_len = int(np.max(ts_len) * context_ratio)
        #if context_ratio > 0:
        #    context_len = context_ratio
        else:    
            context_len = prediction_length*2
        if context_len < 10:
            context_len = 10
        
        print(f'====event:{events[_data[0]]}, prediction_len={prediction_length},train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)},context_len={context_len}')

        #rerank due to short ts removed
        if run_ts == COL_RANK and dorerank == True:
            sel_rows = []
            for rowid in range(_data[2].shape[0]):
                # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
                rec = _data[2][rowid].copy()
                #remove nan(only tails)
                nans, x= nan_helper(rec[run_ts,:])
                nan_count = np.sum(nans)             
                rec = rec[:, ~np.isnan(rec[run_ts,:])]
                
                totallen = rec.shape[1]
                if ( totallen < train_len + prediction_length):
                    print(f'rerank a short ts: carid={_data[1][rowid]}，len={totallen}')
                    continue 
                else:
                    sel_rows.append(rowid)
                    
            #get selected matrix
            sel_idx = np.array(sel_rows)
            selmat = _data[2][sel_idx]
            
            mask = np.isnan(selmat[:,COL_RANK,:])
            
            idx = np.argsort(selmat[:,COL_RANK,:], axis=0)
            true_rank = np.argsort(idx, axis=0).astype(np.float)
            true_rank[mask] = np.nan
            
            #set it back
            #if _data[0]==0:
            #    print('raw:')
            #    print(_data[2][:,COL_RANK,0])
            #    print('true_rank:')
            #    print(true_rank[:,0])
            #_data[2][sel_idx][:,COL_RANK,:] = true_rank       
            _data[2][sel_idx,COL_RANK,:] = true_rank       
            #if _data[0]==0:
            #    _view = _data[2][sel_idx]
            #    _view[:,COL_RANK,:] = true_rank
            #    print('view:')
            #    print(_data[2][:,COL_RANK,0])
            #    print(_view[:,COL_RANK,0])
            #    print('rerank:')
            #    print(_data[2][sel_idx][:,COL_RANK,0])
        
        
        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()
            
            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]
            
            # remove short ts
            totallen = rec.shape[1]
            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                
            
            if use_global_dict:
                carno = _data[1][rowid]
                carid = global_carids[_data[1][rowid]]
            else:
                #simulation dataset, todo, fix the carids as decoder
                carno = rowid
                carid = rowid
                
            
            if useeid:
                static_cat = [carid, _data[0]]    
            else:
                static_cat = [carid]    
                
            #first, get target a copy    
            # target can be COL_XXSTATUS
            target_val = rec[run_ts,:].copy().astype(np.float32)
            if log_transform:
                target_val = np.log(target_val + 1.0)
            
            # selection of features
            if test_flag(oracle_mode, MODE_NOTRACK):                
                rec[COL_TRACKSTATUS, :] = 0
            if test_flag(oracle_mode, MODE_NOLAP):                
                rec[COL_LAPSTATUS, :] = 0

            test_rec_cnt = 0
            if not test_mode:
                # all go to train set
                _train.append({'target': target_val, 
                                'start': start, 
                                'feat_static_cat': static_cat,
                                'feat_dynamic_real': [rec[COL_TRACKSTATUS,:]]
                                #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:],
                                #       rec[COL_LAPSTATUS,:],
                                #       rec[COL_LAPS_INSTINT,:]]
                              }
                              )
            else:
                # reset train_len
                if context_ratio != 0.:
                    # all go to train set
                    #add [0, context_len] to train set 
                    _train.append({'target': target_val[:context_len], 
                                    'start': start, 
                                    'feat_static_cat': static_cat,
                                    'feat_dynamic_real': [rec[COL_TRACKSTATUS,:context_len]]
                                    #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:context_len],
                                    #       rec[COL_LAPSTATUS,:context_len],
                                    #       rec[COL_LAPS_INSTINT,:context_len]               
                                    #                     ]
                                  }
                                  )                    
                
                # testset
                # multiple test ts(rolling window as half of the prediction_length)

                #step = -int(prediction_length/2) if half_moving_win else -prediction_length
                if half_moving_win == 1:
                    step = -int(prediction_length/2)
                elif half_moving_win == 2:
                    step = -prediction_length
                else:
                    step = -1                
                
                for endpos in range(totallen, context_len+prediction_length, 
                                    step):

                    track_rec = rec[COL_TRACKSTATUS, :endpos].copy()
                    lap_rec = rec[COL_LAPSTATUS, :endpos].copy()
                    pitage_rec = rec[COL_LAPS_INSTINT, :endpos].copy()

                    # test mode
                    if test_flag(oracle_mode, MODE_TESTCURTRACK):
                        # since nan does not work, use cur-val instead
                        track_rec[-prediction_length:] = track_rec[-prediction_length - 1]
                        #track_rec[-prediction_length:] = random.randint(0,1)
                        #lap_rec[-prediction_length:] = lap_rec[-prediction_length - 1]
                        lap_rec[-prediction_length:] = 0

                        #for pitage, just assume there is no pit
                        start_pitage = pitage_rec[-prediction_length - 1]
                        pitage_rec[-prediction_length:] = np.array([x+start_pitage+1 for x in range(prediction_length)])

                    elif test_flag(oracle_mode, MODE_TESTZERO):
                        #set prediction part as nan
                        #track_rec[-prediction_length:] = np.nan
                        #lap_rec[-prediction_length:] = np.nan
                        track_rec[-prediction_length:] = 0
                        lap_rec[-prediction_length:] = 0                    

                        #for pitage, just assume there is no pit
                        start_pitage = pitage_rec[-prediction_length - 1]
                        pitage_rec[-prediction_length:] = np.array([x+start_pitage+1 for x in range(prediction_length)])

                    _test.append({'target': rec[run_ts,:endpos].astype(np.float32), 
                                'start': start, 
                                'feat_static_cat': static_cat,
                                'feat_dynamic_real': [track_rec]
                                #'feat_dynamic_real': [track_rec,lap_rec,pitage_rec]
                                #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:endpos],
                                #       rec[COL_LAPSTATUS,:endpos]] 
                                 }
                              )                     
   
                    test_rec_cnt += 1
            
            #add one ts
            print(f'carno:{carno}, totallen:{totallen}, nancount:{nan_count}, test_reccnt:{test_rec_cnt}')

        train_set.extend(_train)
        test_set.extend(_test)

    print(f'train len:{len(train_set)}, test len:{len(test_set)}')
    
    train_ds = ListDataset(train_set, freq=freq)
    test_ds = ListDataset(test_set, freq=freq)    
    
    return train_ds, test_ds, train_set, test_set

def save_dataset(datafile,freq, prediction_length, cardinality, train_ds, test_ds):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [freq, prediction_length, cardinality, train_ds, test_ds]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)

### bulid dataset

In [5]:
def make_gluontsdb_byevent(run_ts, dbname, test_event='Indy500-2018', 
                           log_transform = False,context_ratio=0.,dorerank = True):
    #run on indy dataset
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq,
                                         useeid=useeid, run_ts=run_ts,
                                        test_event=test_event,log_transform =log_transform,
                                        context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-all-all-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    if useeid:
        #special for classic predictor only
        return
    
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOTRACK,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event,
                                        log_transform =log_transform,
                                        context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-notrack-all-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    # nolap+all -> track + pitage
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOLAP,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event,
                                        log_transform =log_transform,
                                        context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-nolap-all-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    
    #pitage only
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, prediction_length,freq, 
                                         oracle_mode=MODE_NOTRACK+MODE_NOLAP,useeid=useeid, 
                                         run_ts=run_ts,test_event=test_event,
                                         log_transform =log_transform,
                                         context_ratio=context_ratio,dorerank =dorerank)
    save_dataset(f'{dbname}-oracle-{ipstr}-zero-zero-f{freq}-t{prediction_length}-r{test_event}-gluonts-indy-2018.pickle'
                 , freq, prediction_length, cardinality,train_ds, test_ds)
    

### create dbs

In [10]:
useeid = False
interpolate = False
ipstr =''
cardinality=[]
prediction_length=0
def makedbs():
    global useeid,interpolate,ipstr,cardinality,prediction_length
    
    useeid = False
    interpolate = False
    #ipstr = '-ip' if interpolate else '-noip'
    ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
    if useeid:
        cardinality = [len(global_carids), len(laptime_data)]
    else:
        cardinality = [len(global_carids)]

    #plens=[2,5,10,20,30]
    #plens=[1, 2,5,10]
    plens=[1]
    for prediction_length in plens:
        make_gluontsdb_byevent(_run_ts,_task_id,context_ratio=context_ratio)
        
    return

    #special
    interpolate = True
    useeid = False
    ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
    for prediction_length in plens:
        make_gluontsdb_byevent(_run_ts,_task_id,context_ratio=context_ratio)

    useeid = True
    interpolate = False
    #ipstr = '-ip' if interpolate else '-noip'
    ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
    if useeid:
        cardinality = [len(global_carids), len(laptime_data)]
    else:
        cardinality = [len(global_carids)]

    for prediction_length in plens:
        make_gluontsdb_byevent(_run_ts,_task_id,context_ratio=context_ratio)    

In [7]:
def makedir(outdir):
    if not os.path.exists(outdir):
        os.mkdir(outdir)

In [8]:
#years = ['2013','2014','2015','2016','2017','2018','2019']
years = ['2013','2014','2015','2016','2017','2018']
#events = ['Indy500']
events = [f'Indy500-{x}' for x in years]
events_id={key:idx for idx, key in enumerate(events)}
dbid = f'Indy500_{years[0]}_{years[-1]}_v9'
# start from here
import pickle
#with open('laptime_rank_timediff_fulltest-oracle-%s.pickle'%year, 'rb') as f:
with open(f'laptime_rank_timediff_pit-oracle-{dbid}.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    global_carids, laptime_data = pickle.load(f, encoding='latin1') 

In [13]:
### split by event -- laptime

freq = "1min"
context_ratio=0.
#context_ratio=0.2
#context_ratio=0.4

outdir = 'nextpitmodel'
makedir(outdir)

subdir = 'nextpit-indy500'
makedir(f'{outdir}/{subdir}' )
_run_ts = COL_LAP2NEXTPIT
_task_id = f'{outdir}/{subdir}/nextpit'
makedbs()


====event:Indy500-2013, prediction_len=1,train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=0
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=30
carno:7, totallen:177, nancount:23, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:187, nancount:13, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:170, nancount:30, test_reccnt:0
carno:16, totallen:179, nancount:21, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:170, nancount:30, test_reccnt:0
car

carno:22, totallen:200, nancount:0, test_reccnt:0
carno:25, totallen:200, nancount:0, test_reccnt:0
carno:26, totallen:200, nancount:0, test_reccnt:0
carno:27, totallen:169, nancount:31, test_reccnt:0
carno:28, totallen:200, nancount:0, test_reccnt:0
carno:33, totallen:200, nancount:0, test_reccnt:0
carno:34, totallen:200, nancount:0, test_reccnt:0
carno:41, totallen:187, nancount:13, test_reccnt:0
carno:63, totallen:184, nancount:16, test_reccnt:0
a short ts: carid=67，len=122
carno:68, totallen:200, nancount:0, test_reccnt:0
carno:77, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=83，len=148
a short ts: carid=91，len=86
carno:98, totallen:200, nancount:0, test_reccnt:0
====event:Indy500-2015, prediction_len=1,train_len=160, max_len=200, min_len=200,context_len=10
carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
carno:4, totallen:169, nancount:31, test_reccnt:0
carno:5, totallen:

carno:1, totallen:200, nancount:0, test_reccnt:0
carno:2, totallen:200, nancount:0, test_reccnt:0
carno:3, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=4，len=0
carno:5, totallen:200, nancount:0, test_reccnt:0
a short ts: carid=6，len=30
carno:7, totallen:177, nancount:23, test_reccnt:0
carno:8, totallen:200, nancount:0, test_reccnt:0
carno:9, totallen:200, nancount:0, test_reccnt:0
carno:10, totallen:187, nancount:13, test_reccnt:0
carno:11, totallen:200, nancount:0, test_reccnt:0
carno:12, totallen:200, nancount:0, test_reccnt:0
carno:14, totallen:200, nancount:0, test_reccnt:0
carno:15, totallen:170, nancount:30, test_reccnt:0
carno:16, totallen:179, nancount:21, test_reccnt:0
carno:18, totallen:200, nancount:0, test_reccnt:0
carno:19, totallen:200, nancount:0, test_reccnt:0
carno:20, totallen:200, nancount:0, test_reccnt:0
carno:21, totallen:170, nancount:30, test_reccnt:0
carno:22, totallen:200, nancount:0, test_reccnt:0
carno:25, totallen:200, nancount:0, test_reccnt:0

In [ ]:
sys.exit(0)

In [ ]:

freq = "1min"
context_ratio=0.
#context_ratio=0.2
#context_ratio=0.4

outdir = 'pitagemodel'
makedir(outdir)

subdir = 'pitage-indy500'
makedir(f'{outdir}/{subdir}' )
_run_ts = COL_LAPS_INSTINT
_task_id = f'{outdir}/{subdir}/pitage'
makedbs()

### test

In [ ]:
def test_a(a, bitflag):
    return (a & bitflag) ==  bitflag

test_a(5, 4)


In [ ]:
mat = np.random.randint(5, size=(10,2, 5)).astype(np.float)
mat[2,1,0] = np.nan
_idx = np.array([0,1,2,8])
selmat = mat[_idx]
selmat[:,1,:]

In [ ]:
mask = np.isnan(selmat[:,1,:])
mask

In [ ]:
true_rank.shape

In [ ]:
idx = np.argsort(selmat[:,1,:], axis=0)
true_rank = np.argsort(idx, axis=0).astype(np.float)
true_rank[mask] = np.nan
true_rank

In [ ]:
selmat[:,1,:] = true_rank
selmat

In [ ]:
mat[_idx]

In [ ]:
mat[_idx] = selmat

In [ ]:
mat[_idx]

In [ ]:
freq, prediction_length, cardinality = 0,0,0

def load_dataset(inputfile):
    global freq, prediction_length, cardinality

    with open(inputfile, 'rb') as f:
        # have to specify it.
        freq, prediction_length, cardinality,train_ds, test_ds = pickle.load(f, encoding='latin1')
    
    print(f"number of cars: {cardinality}")
    
    return train_ds, test_ds

train_ds, test_ds = load_dataset('rank-oracle-noip-eid-all-all-f1min-t30-rIndy500-gluonts-indy-2018.pickle')

In [ ]:
ds_iter =  iter(test_ds)
rec = next(ds_iter)
rec

In [ ]:
def debug_get_ranklist(train_ds):
    ds_iter =  iter(train_ds)
    ranklist = []
    for idx in range(len(train_ds)):
        rec = next(ds_iter)

        if rec['feat_static_cat'][1] == 0:
            ranklist.append(rec['target'][0])

    return np.array(sorted(ranklist))

In [ ]:
len(ranklist)

In [ ]:
def get_testds(dorerank):
    train_ds, test_ds,_,_ = make_dataset_byevent(-1, 2,freq,
                                        useeid=True, run_ts=COL_RANK,
                                        test_event='Indy500',log_transform =False,
                                        context_ratio=0,
                                                dorerank=dorerank)
    return train_ds, test_ds

In [ ]:
rank_train, rank_ds = get_testds(True)

In [ ]:
rawrank_train, rawrank_ds = get_testds(False)

In [ ]:
debug_get_ranklist(rank_train)

In [ ]:
debug_get_ranklist(rawrank_train)

In [ ]:
debug_get_ranklist(train_ds)

In [ ]:
 #alldata, rankdata, acldata, flagdata = stagedata['Indy500-2011']

In [ ]:
set(acldata.car_number.values)

In [ ]:
acldata

In [ ]:
acldata[acldata['completed_laps']==187]

In [ ]:
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==186].car_number.values))))
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==187].car_number.values))))
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==188].car_number.values))))
print(','.join(list(sorted(rankdata[rankdata['completed_laps']==189].car_number.values))))


In [ ]:
alldata, rankdata, acldata, flagdata = stagedata['Indy500-2014']

In [ ]:
kannan = rankdata[rankdata['car_number']==10]
kannan[(kannan['completed_laps'] > 60) & (kannan['completed_laps']<70)]

In [ ]:
alldata, rankdata, acldata, flagdata = stagedata['Indy500-2013']
rankdata[(rankdata['car_number']==16) & (rankdata['completed_laps']<10)]

In [ ]:
rankdata[(rankdata['car_number']==16) & (rankdata['completed_laps']>88)& (rankdata['completed_laps']<97)]

In [ ]:
car16 = acldata[acldata['car_number']==16]


In [ ]:
for idx in range(len(car16)):
    if(car16.iloc[idx].lap_status == 'P'):
        print(car16.iloc[idx].completed_laps)

In [ ]:
for idx in range(len(car16)):
    if(car16.iloc[idx].track_status == 'Y'):
        print(car16.iloc[idx].completed_laps)

In [ ]:
alldata, rankdata, acldata, flagdata = stagedata['Indy500-2014']
car16 = acldata[acldata['car_number']==14]
car16[car16['lap_status']=='P']

### check the laptime data

In [ ]:
#Indy500-2018 car 12 -> eventid = 5(-2), carid = 7
#LAP_STATUS = 3
lapstatus = laptime_data[-2][2][7][3].astype(np.int)
lapstatus

In [ ]:
np.where(lapstatus==1)[0]

In [ ]:
#LAPS_INSTINT = 6,
pitage = laptime_data[-2][2][7][6].astype(np.int)
pitage

In [ ]:
np.where(pitage==0)[0]

In [ ]:
#CAUTION_LAPS_INSTINT = 5
trackstatus = laptime_data[-2][2][7][2].astype(np.int)
trackstatus

In [ ]:
np.where(trackstatus==1)[0]

In [ ]:
#CAUTION_LAPS_INSTINT = 5
yfage = laptime_data[-2][2][7][5].astype(np.int)
yfage 

In [ ]:
np.where(yfage!=0)[0]

In [ ]:
outdir = 'pitage'
if not os.path.exists(outdir):
    os.mkdir(outdir)


In [11]:
def load_dataset(inputfile):
    global freq, prediction_length, cardinality

    with open(inputfile, 'rb') as f:
        # have to specify it.
        freq, prediction_length, cardinality,train_ds, test_ds = pickle.load(f, encoding='latin1')
    
    print(f"number of cars: {cardinality}")
    
    return train_ds, test_ds

In [12]:
train_ds, test_ds = load_dataset('pitage/laptime-indy500/laptime-oracle-ip-noeid-nolap-zero-f1min-t2-rIndy500-2018-gluonts-indy-2018.pickle')

FileNotFoundError: [Errno 2] No such file or directory: 'pitage/laptime-indy500/laptime-oracle-ip-noeid-nolap-zero-f1min-t2-rIndy500-2018-gluonts-indy-2018.pickle'

In [ ]:
ds_iter =  iter(train_ds)
#ranklist = []
#for idx in range(len(train_ds)):
#    rec = next(ds_iter)
rec = next(ds_iter)
rec

In [ ]:
ds_iter =  iter(test_ds)
#ranklist = []
#for idx in range(len(train_ds)):
#    rec = next(ds_iter)
rec = next(ds_iter)
rec

In [ ]:
events

In [ ]:
test_event

In [29]:
data = laptime_data[-1][2]
data.shape

(33, 8, 200)

In [46]:
lapstatusA = laptime_data[-1][2][1,3,:]
lapstatusA = laptime_data[-1][2][0,3,:]

In [47]:
lapstatusA

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [48]:
lap2nextpitA = get_lap2nextpit(lapstatusA)
lap2nextpitA

array([31., 30., 29., 28., 27., 26., 25., 24., 23., 22., 21., 20., 19.,
       18., 17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,
        5.,  4.,  3.,  2.,  1., 18., 17., 16., 15., 14., 13., 12., 11.,
       10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1., 20., 19., 18.,
       17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,
        4.,  3.,  2.,  1., 31., 30., 29., 28., 27., 26., 25., 24., 23.,
       22., 21., 20., 19., 18., 17., 16., 15., 14., 13., 12., 11., 10.,
        9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1., 34., 33., 32., 31.,
       30., 29., 28., 27., 26., 25., 24., 23., 22., 21., 20., 19., 18.,
       17., 16., 15., 14., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,
        4.,  3.,  2.,  1., 35., 34., 33., 32., 31., 30., 29., 28., 27.,
       26., 25., 24., 23., 22., 21., 20., 19., 18., 17., 16., 15., 14.,
       13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1.,
       31., 30., 29., 28., 27., 26., 25., 24., 23., 22., 21., 20

In [49]:
pitstops = np.where(lapstatusA==1)[0]
pitstops

array([ 31,  49,  69, 100, 134, 169])

In [50]:
len(lap2nextpitA)

200

In [51]:
nans, x= nan_helper(lapstatusA)
np.sum(nans)   

0

In [52]:
nans, x= nan_helper(lap2nextpitA)
np.sum(nans)  

0